### Chat History with Memory 🧠

<img src="./Image/ChatHistory.png" width="700" height="400" style="display: block; margin: auto;">

### Load ENV file

In [ ]:
from dotenv import load_dotenv
import os
load = load_dotenv('./../.env')

print(os.getenv("LANGSMITH_PROJECT"))

#### Create LLM object

In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    base_url="http://localhost:11434",
    model = "qwen2.5:latest",
    temperature=0.5,
    max_tokens = 250
)

In [ ]:
#!pip install langchain_community

### Message history with ChatMessageHistory

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import chain
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory


In [ ]:
template = ChatPromptTemplate.from_messages([
     ('placeholder', "{history}"),
    ('human', "{prompt}")
])

chain = template | llm | StrOutputParser()

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


history = RunnableWithMessageHistory(
    chain, 
    get_session_history,
    input_messages_key="prompt",
    history_messages_key="history")

session_id = "Karthik"

get_session_history(session_id).clear()

response1 = history.invoke({"prompt": "What is the distance between earth and sun?"},
                          config = {"configurable": {"session_id": session_id}})

response2 = history.invoke({"prompt": "How about from moon"},
                          config = {"configurable": {"session_id": session_id}})

print(response1)
print("\n\n")
print(response2)



### ChatMessage History with SqlChatMessageHistory

In [ ]:
from langchain_community.chat_message_histories import SQLChatMessageHistory


def get_session_history(session_id):
    return SQLChatMessageHistory(session_id=session_id, connection_string="sqlite:///chathistory.db")

template = ChatPromptTemplate.from_messages([
    ('placeholder', "{history}"),
    ('human', "{prompt}")
])

chain = template | llm | StrOutputParser()

history = RunnableWithMessageHistory(
    chain, 
    get_session_history,
    input_messages_key="prompt",
    history_messages_key="history")


session_id = "Karthik"

get_session_history(session_id).clear()

response1 = history.invoke({"prompt": "What is the distance between earth and sun?"},
                          config = {"configurable": {"session_id": session_id}})

response2 = history.invoke({"prompt": "How about from moon"},
                          config = {"configurable": {"session_id": session_id}})

print(response1)
print("\n\n")
print(response2)

